In [1]:
import sys, os
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))
from wofscast.data_generator import (add_local_solar_time, 
                                     to_static_vars, 
                                     dataset_to_input,
                                     #load_chunk,
                                     ZarrDataGenerator
                                    )
from wofscast import data_utils
from wofscast.wofscast_task_config import DBZ_TASK_CONFIG, WOFS_TASK_CONFIG
from os.path import join

import xarray as xr
from dask.distributed import Client

In [2]:
%%time
import os
from os.path import join
from concurrent.futures import ThreadPoolExecutor

base_path = '/work/mflora/wofs-cast-data/datasets_zarr'#_zarr'
years = ['2019', '2020']

def get_files_for_year(year):
    year_path = join(base_path, year)
    with os.scandir(year_path) as it:
        return [join(year_path, entry.name) for entry in it if entry.is_dir() and entry.name.endswith('.zarr')]
        #return [join(year_path, entry.name) for entry in it if entry.is_file()]
    
with ThreadPoolExecutor() as executor:
    paths = []
    for files in executor.map(get_files_for_year, years):
        paths.extend(files)

print(len(paths))

18053
CPU times: user 35.5 ms, sys: 25.1 ms, total: 60.5 ms
Wall time: 39.3 ms


In [3]:
paths[0]

'/work/mflora/wofs-cast-data/datasets_zarr/2019/wrfwof_2019-05-16_040000_to_2019-05-16_043000__10min__ens_mem_07.zarr'

In [4]:
%%time 

generator = ZarrDataGenerator(DBZ_TASK_CONFIG, cpu_batch_size=64, gpu_batch_size=32, n_workers=16)

j=0
for inputs, targets, forcings in generator(paths[:1024]):
    print(f'Batch : {j}')
    j+=1
   
# DBZ_TASK_CONFIG, threading = 25.6 seconds (2 workers)
# DBZ_TASK_CONFIG, threading = 23.7 seconds (1 worker)
# DBZ_TASK_CONFIG, no threading = 23.1 seconds

/work/mflora/miniconda3/envs/wofs-cast/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/work/mflora/miniconda3/envs/wofs-cast/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Batch : 0
Batch : 1
Batch : 2
Batch : 3
Batch : 4
Batch : 5
Batch : 6
Batch : 7
Batch : 8
Batch : 9
Batch : 10
Batch : 11
Batch : 12
Batch : 13
Batch : 14
Batch : 15
Batch : 16
Batch : 17
Batch : 18
Batch : 19
Batch : 20
Batch : 21
Batch : 22
Batch : 23
Batch : 24
Batch : 25
Batch : 26
Batch : 27
Batch : 28
Batch : 29
Batch : 30
Batch : 31
CPU times: user 4.6 s, sys: 1.45 s, total: 6.05 s
Wall time: 6.63 s
